In [1]:
import string, re
import nltk
from nltk import word_tokenize
import pandas as pd
import numpy as np

In [2]:
def tokens(texto):
    """Retorna una lista con los tokens de texto"""
    texto = texto.lower() #convevrtir a minuscula
    texto = re.sub(r'\d+', '', texto)
    texto = texto.translate(str.maketrans('','', string.punctuation)) #eliminar puntuacion
    #texto = texto.strip() #remove whitspaces at begin and end
    return word_tokenize(texto)

In [4]:
#falta ignorar espacios
file = "400finito.txt"
texto = ""
with open(file, encoding='latin-1') as csvfile:
    texto = csvfile.read()
    csvfile.seek(0)
    lineas = csvfile.readlines()

In [5]:
tokens_texto= tokens(texto)
texto_fd = nltk.FreqDist(tokens_texto)

In [8]:
len(lineas) == 400*16

True

In [18]:
400*16

6400

## Por categoría (n=5)

In [27]:
n=16 #numero de categorías
masc = '1' #masculino
fem = '2' #femenino
data_linea = {i:[] for i in range(1,n+1)} #Todos los datos por categoría
data_linea_raw = {i:[] for i in range(1,n+1)} #Todos los datos por categoría texto
j=0
code_sexo= {'1':'masculino', '2':'femenino'}
code_colegio = {'1':'publico', '2':'pagado', '3':'subencionado'}
for i,linea in enumerate(lineas):
    if i%n==0:
        j=0
    #code = linea.split()[0][0]
    #if code == fem:
    data_linea[j+1].append(tokens(linea)) #tokens por linea
    data_linea_raw[j+1].append(re.sub(r'\d+', '', linea)) #texto por linea
    j+=1

In [30]:
len(data_linea[16])

400

In [32]:
data_cat ={i:[] for i in range(1,n+1)}  #tokens para FreqDist
for categoria in data_linea:
    data_i = [] #tokens por categoría
    for linea in data_linea[categoria]:
        data_i+=linea
    data_cat[categoria] += data_i

In [33]:
freqdist_cat = {i:[] for i in range(1,n+1)} #Frecuencias de tokens
for cat in data_cat:
    freqdist_cat[cat] = nltk.FreqDist(data_cat[cat])

## n-grams

In [35]:
from nltk.util import ngrams
import networkx as nx

DataFrame para cada categoría con bigramas para plotear grafo

In [36]:
data_cat_ngrams = {} #ngrams para cada categoría
data_cat_fd = {} #FreqDist para cada categoria
for cat in data_cat:
    n_grams = ngrams(data_cat[cat], 2)
    data_cat_ngrams[cat] = [ ' '.join(grams) for grams in n_grams]
    data_cat_fd[cat] = nltk.FreqDist(data_cat_ngrams[cat])

DataFrame para cada categoría con bigramas para plotear grafo

In [37]:
data_cat_fd_dataframe = {} #dataframe con frecuencias de bigrams para cada categoria
for k in data_cat_fd:
    fd_sorted = {k: v for k, v in sorted(data_cat_fd[k].items(), key=lambda item: item[1], reverse=True)}
    keys = list(fd_sorted.keys())
    freq = list(fd_sorted.values())
    origen = [bigram.split()[0] for bigram in keys]
    destino = [bigram.split()[1] for bigram in keys]
    #n = list(data_cat_fd[k].values())
    data = {
        "origen": origen,
        "destino": destino,
        "n": freq
    }
    data_cat_fd_dataframe[k] = pd.DataFrame(data) #dataframe por categoria para crear grafo 

In [42]:
len(data_cat_fd_dataframe)

16

In [44]:
nombres_categorias = [
    "Partes del cuerpo",
    "La ropa",
    "Partes de la casa (sin los muebles)",
    "Los muebles",
    "Alimentos y bebidas",
    "Objetos colocados encima de la mesa para la comida",
    "La cocina y sus utensilios",
    "La escuela",
    "Calefacción, iluminación y medios para airear un recinto",
    "La ciudad",
    "El campo",
    "Medios de transporte",
    "Trabajos del campo y del jardín",
    "Animales",
    "Juegos y distracciones",
    "Profesiones y oficio"
]

In [45]:
"Partes del cuerpo".replace(' ', '_')

'Partes_del_cuerpo'

In [40]:
from graphviz import *

In [46]:
for centro in data_cat_fd:
    origen_ = list(data_cat_fd_dataframe[centro].origen)
    destino_ = list(data_cat_fd_dataframe[centro].destino)
    n_ = list(data_cat_fd_dataframe[centro].n)
    nombre = nombres_categorias[centro-1].replace(' ', '_')
    dot = Digraph(name=nombre, comment='Grafo dirigido palabra por categoria')
    for o,d,i in zip(origen_, destino_, n_):
        #if i>5:
        dot.edge(o, d, label=str(i), weight=str(i))
    #dot.attr(bgcolor='purple:pink', label='Categoría 1', fontcolor='white')
    dot.save()